In [1]:
from pydantic import BaseModel

In [2]:
class User(BaseModel):
    id: int
    name = "Jane Doe"

In [3]:
user = User(id='123')
user_x = User(id=123.45)

In [4]:
assert user.id == 123
assert user_x.id == 123
assert isinstance(user_x.id, int)

In [5]:
assert user.name == "Jane Doe"

In [6]:
assert user.__fields_set__ == {"id"}

In [7]:
assert user.dict() == dict(user) == {"id": 123, "name": "Jane Doe"}

In [8]:
user.id = 321
assert user.id == 321